# Herb Image Classifier Project
This notebook demonstrates a deep learning image classifier for herbs. It predicts the herb's name from an input image and displays its region and medicinal uses.

In [1]:
# Sample dataset for herbs
herb_data = {
    "Tulsi": {
        "region": "North & Central India",
        "uses": "Boosts immunity, reduces stress, anti-inflammatory"
    },
    "Neem": {
        "region": "All over India, mostly tropical regions",
        "uses": "Skin treatment, antibacterial, purifies blood"
    },
    "AloeVera": {
        "region": "South India, arid regions",
        "uses": "Skin care, digestive aid, wound healing"
    }
}
import pandas as pd
herb_df = pd.DataFrame.from_dict(herb_data, orient='index')
herb_df.reset_index(inplace=True)
herb_df.rename(columns={'index': 'herbs'}, inplace=True)
herb_df

,herbs,region,uses
0,Tulsi,North & Central India,"Boosts immunity, reduces stress, anti-inflamma..."
1,Neem,"All over India, mostly tropical regions","Skin treatment, antibacterial, purifies blood"
2,AloeVera,"South India, arid regions","Skin care, digestive aid, wound healing"


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path to dataset
train_dir = "herb_dataset/train"
test_dir = "herb_dataset/test"

# Image preprocessing
datagen = ImageDataGenerator(rescale=1./255)

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=2,
    class_mode='categorical'
    )

test_data = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=2,
    class_mode='categorical'
    )

print("Classes:", train_data.class_indices)

Found 1615 images belonging to 3 classes.
Found 48 images belonging to 3 classes.
Classes: {'Aloe Vera': 0, 'Neem': 1, 'Tulsi': 2}


In [3]:
# Install TensorFlow in notebook environment
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [4]:
# Define a lightweight CNN model for herb classification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax') # 3 classes: AloeVera, Neem, Tulsi
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

d:\anaconda\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,747 (92.76 KB)

 Trainable params: 23,747 (92.76 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# -------------------
# Paths (Update if needed)
# -------------------
train_dir = "herb_dataset/train"
val_dir   = "herb_dataset/test"

# -------------------
# Data Augmentation
# -------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

# -------------------
# Load Pretrained MobileNetV2
# -------------------
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Unfreeze last layers for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:100]:   # freeze first 100 layers
    layer.trainable = False

# -------------------
# Custom Layers on top
# -------------------
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.4)(x)
predictions = Dense(train_generator.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

# -------------------
# Compile
# -------------------
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

# -------------------
# Callbacks
# -------------------
early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", monitor="val_accuracy", save_best_only=True)

# -------------------
# Training
# -------------------
history = model.fit(
    train_generator,
    epochs=40,   # go longer, early stopping will handle overfit
    validation_data=val_generator,
    callbacks=[early_stop, checkpoint]
)

# -------------------
# Save final model
# -------------------
model.save("final_herb_model.h5")
print("✅ Training done. Model saved as final_herb_model.h5")

# -------------------
# Evaluation & Confusion Matrix
# -------------------
loss, acc = model.evaluate(val_generator)
print(f"Test Accuracy: {acc*100:.2f}%")

Y_pred = model.predict(val_generator)
y_pred = np.argmax(Y_pred, axis=1)

print("\nConfusion Matrix:")
print(confusion_matrix(val_generator.classes, y_pred))

print("\nClassification Report:")
target_names = list(val_generator.class_indices.keys())
print(classification_report(val_generator.classes, y_pred, target_names=target_names))


Found 1615 images belonging to 3 classes.
Found 48 images belonging to 3 classes.


d:\anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4812 - loss: 1.0843

51/51 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.5870 - loss: 0.9107 - val_accuracy: 0.5417 - val_loss: 0.8438
Epoch 2/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8364 - loss: 0.4963

51/51 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.8663 - loss: 0.4253 - val_accuracy: 0.5833 - val_loss: 0.6924
Epoch 3/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9432 - loss: 0.2616

...

51/51 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.9944 - loss: 0.0410 - val_accuracy: 0.8958 - val_loss: 0.2981
Epoch 10/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.9944 - loss: 0.0345 - val_accuracy: 0.8958 - val_loss: 0.2652
Epoch 11/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.9944 - loss: 0.0313 - val_accuracy: 0.8958 - val_loss: 0.2294
Epoch 12/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.9988 - loss: 0.0198 - val_accuracy: 0.8958 - val_loss: 0.2108
Epoch 13/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 103s 2s/step - accuracy: 0.9969 - loss: 0.0209 - val_accuracy: 0.8958 - val_loss: 0.1856
Epoch 14/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 113s 2s/step - accuracy: 0.9994 - loss: 0.0143 - val_accuracy: 0.8958 - val_loss: 0.1660
Epoch 15/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9984 - loss: 0.0179

51/51 ━━━━━━━━━━━━━━━━━━━━ 111s 2s/step - accuracy: 0.9981 - loss: 0.0182 - val_accuracy: 0.9167 - val_loss: 0.1514
Epoch 16/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9986 - loss: 0.0159

51/51 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 0.9975 - loss: 0.0157 - val_accuracy: 0.9375 - val_loss: 0.1381
Epoch 17/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 851ms/step - accuracy: 0.9975 - loss: 0.0125

51/51 ━━━━━━━━━━━━━━━━━━━━ 45s 872ms/step - accuracy: 0.9975 - loss: 0.0137 - val_accuracy: 0.9583 - val_loss: 0.1223
Epoch 18/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 46s 891ms/step - accuracy: 0.9981 - loss: 0.0117 - val_accuracy: 0.9583 - val_loss: 0.1117
Epoch 19/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 51s 993ms/step - accuracy: 0.9994 - loss: 0.0100 - val_accuracy: 0.9583 - val_loss: 0.1073
Epoch 20/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 48s 949ms/step - accuracy: 1.0000 - loss: 0.0079 - val_accuracy: 0.9583 - val_loss: 0.1091
Epoch 21/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 48s 939ms/step - accuracy: 0.9975 - loss: 0.0089 - val_accuracy: 0.9583 - val_loss: 0.1099
Epoch 22/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.9988 - loss: 0.0095 - val_accuracy: 0.9375 - val_loss: 0.1164
Epoch 23/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 49s 948ms/step - accuracy: 0.9994 - loss: 0.0062 - val_accuracy: 0.9375 - val_loss: 0.1231
Epoch 24/40
51/51 ━━━━━━━━━━━━━━━━━━━━ 50s 978ms/step - accuracy: 0.9994 - loss: 0.0071 - val_accuracy: 0

✅ Training done. Model saved as final_herb_model.h5
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 234ms/step - accuracy: 0.9583 - loss: 0.1073
Test Accuracy: 95.83%
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step

Confusion Matrix:
[[12  0  0]
 [ 0 16  0]
 [ 1  1 18]]

Classification Report:
              precision    recall  f1-score   support

   Aloe Vera       0.92      1.00      0.96        12
        Neem       0.94      1.00      0.97        16
       Tulsi       1.00      0.90      0.95        20

    accuracy                           0.96        48
   macro avg       0.95      0.97      0.96        48
weighted avg       0.96      0.96      0.96        48



In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Recreate the generator (same settings as before, only for mapping)
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    "herb_dataset/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

# Map back to class labels
class_labels = {v: k for k, v in train_generator.class_indices.items()}
print("Class Labels Mapping:", class_labels)


Found 1615 images belonging to 3 classes.
Class Labels Mapping: {0: 'Aloe Vera', 1: 'Neem', 2: 'Tulsi'}


In [12]:
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model

model = load_model("final_herb_model.h5")

def predict_herb(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction)
    return class_labels[class_idx]

print(predict_herb("download (1).jpeg"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 717ms/step
Tulsi
